In [1]:
%load_ext tensorboard

In [2]:
from pandas import read_parquet
from data import file

data_train = read_parquet(file.news_articles_cleaned_train)
data_test = read_parquet(file.news_articles_cleaned_test)
data_val = read_parquet(file.news_articles_cleaned_val)

In [3]:
data_train

,text_original,label,text_tokenized,text_tokenized_keywords,text_keywords,text_tokenized_lemmas,text_lemmas,text_tokenized_stemmed,text_stem
1637,"Asylverfahren sollen verkürzt werden, dafür is...",Wirtschaft,"[asylverfahren, sollen, verkürzt, werden, ,, d...","[asylverfahren, sollen, verkürzt, ,, dafür, ge...","asylverfahren sollen verkürzt , dafür gewerksc...","[asylverfahren, sollen, verkürzen, dafür, gewe...",asylverfahren sollen verkürzen dafür gewerksch...,"[asylverfahr, soll, verkurzt, dafur, gewerksch...",asylverfahr soll verkurzt dafur gewerkschaft b...
7445,Umsatz im abgelaufenen Quartal fiel um 8 Proze...,Web,"[umsatz, im, abgelaufenen, quartal, fiel, um, ...","[umsatz, abgelaufenen, quartal, fiel, 8, proze...","umsatz abgelaufenen quartal fiel 8 prozent 12,...","[umsatz, abgelaufen, quartal, fallen, 8, proze...","umsatz abgelaufen quartal fallen 8 prozent 12,...","[umsatz, abgelauf, quartal, fiel, 8, prozent, ...","umsatz abgelauf quartal fiel 8 prozent 12,9 mr..."
8808,Wegen Nichterfüllung eines Lizenzkriteriums in...,Sport,"[wegen, nichterfüllung, eines, lizenzkriterium...","[wegen, nichterfüllung, lizenzkriteriums, stad...",wegen nichterfüllung lizenzkriteriums stadionf...,"[wegen, nichterfüllung, lizenzkriteriums, stad...",wegen nichterfüllung lizenzkriteriums stadionf...,"[weg, nichterfull, lizenzkriterium, stadionfra...",weg nichterfull lizenzkriterium stadionfrag na...
1982,Demokratische Präsidentschaftsanwärterin forde...,International,"[demokratische, präsidentschaftsanwärterin, fo...","[demokratische, präsidentschaftsanwärterin, fo...",demokratische präsidentschaftsanwärterin forde...,"[demokratische, präsidentschaftsanwärterin, fo...",demokratische präsidentschaftsanwärterin forde...,"[demokrat, prasidentschaftsanwarterin, fordert...",demokrat prasidentschaftsanwarterin fordert ko...
255,Republikaner: Transparenzregeln umgangen. Die ...,Web,"[republikaner, :, transparenzregeln, umgangen,...","[republikaner, :, transparenzregeln, umgangen,...",republikaner : transparenzregeln umgangen . us...,"[republikaner, transparenzregeln, umgangen, us...",republikaner transparenzregeln umgangen us-prä...,"[republikan, transparenzregeln, umgang, us-pra...",republikan transparenzregeln umgang us-praside...
...,...,...,...,...,...,...,...,...,...
5431,"Premier Alexis Tsipras, der ein neuerliches Vo...",International,"[premier, alexis, tsipras, ,, der, ein, neuerl...","[premier, alexis, tsipras, ,, neuerliches, vot...","premier alexis tsipras , neuerliches votum aus...","[premier, alexis, tsipras, neuerlich, votum, a...",premier alexis tsipras neuerlich votum ausschl...,"[premi, alexis, tsipras, neu, votum, ausgeschl...",premi alexis tsipras neu votum ausgeschloss st...
2368,Ein Kreditereignis bei BMW sollte ausbleiben. ...,Wirtschaft,"[ein, kreditereignis, bei, bmw, sollte, ausble...","[kreditereignis, bmw, ausbleiben, ., bmw, bedi...",kreditereignis bmw ausbleiben . bmw bedient ma...,"[kreditereignis, bmw, ausbleiben, bmw, bediene...",kreditereignis bmw ausbleiben bmw bedienen mar...,"[kreditereignis, bmw, ausbleib, bmw, bedient, ...",kreditereignis bmw ausbleib bmw bedient markt ...
3642,Fledermäuse sind von vielen Krankheitserregern...,Wissenschaft,"[fledermäuse, sind, von, vielen, krankheitserr...","[fledermäuse, vielen, krankheitserregern, ,, s...","fledermäuse vielen krankheitserregern , säuget...","[fledermäuse, viel, krankheitserregern, säuget...",fledermäuse viel krankheitserregern säugetiere...,"[fledermaus, viel, krankheitserreg, saugeti, b...",fledermaus viel krankheitserreg saugeti befall...
8420,"Ein Fossil aus Spanien mit ""Star Wars""-Bezug. ...",Wissenschaft,"[ein, fossil, aus, spanien, mit, ``, star, war...","[fossil, spanien, ``, star, wars, '', -bezug, ...",fossil spanien `` star wars '' -bezug . madrid...,"[fossil, spanien, ``, star, wars, -bezug, madr...",fossil spanien `` star wars -bezug madrid wien...,"[fossil, spani, ``, star, war, '', -bezug, mad...",fossil spani `` star war '' -bezug madrid/w

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, InputLayer
from tensorflow.keras import Sequential
import os
import datetime
from tensorflow import keras
from keras.callbacks import TensorBoard
from preprocessing.text import extract_vocabulary
from models.text.layers import create_text_vectorization, create_word_embedding
from sklearn.preprocessing import LabelBinarizer
    
    
def build_model(X_train, y_train, conv_num_filters=128, conv_kernel_size=7):

    vocabulary, embedding_length = extract_vocabulary(X_train, verbose=True)
    vectorize_layer = create_text_vectorization(vocabulary)
    embedding_layer = create_word_embedding(vocabulary, embedding_length)

    output_classes = len(label_binarizer.classes_)
    
    model = Sequential(name="cnn")
    model.add(InputLayer(input_shape=(1,), dtype=tf.string, name="text_input"))
    model.add(vectorize_layer)
    model.add(embedding_layer)
    model.add(Conv1D(conv_num_filters, conv_kernel_size, activation="relu", strides=1, padding="valid", name="conv_1"))
    model.add(GlobalMaxPooling1D(name="global_max_pool_1"))
    model.add(Dense(output_classes, activation=tf.nn.softmax, name="prediction"))

    return (model)

In [5]:
X_train = data_train.text_tokenized_stemmed
y_train = data_train.label

X_test = data_test.text_tokenized_stemmed
y_test = data_test.label

In [6]:
from preprocessing.categorical import binarizer
label_binarizer = binarizer(y_train)
model = build_model(X_train, y_train)
model.summary()

Median sequence length:       : 173
Percentil                     : 0.98)
Cutoff sequence length        : 594
Max sequence length           : 2590
Embedding length              : 594
Vocabulary length             : 138905


2021-11-18 01:42:15.731333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 01:42:15.739915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 01:42:15.740559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 01:42:15.741586: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

MemoryError: std::bad_alloc

In [ ]:
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

learning_rate=0.01
model_metric = ["accuracy"]
loss_function = CategoricalCrossentropy()
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss=loss_function, optimizer=optimizer, metrics=model_metric)

In [ ]:
data_train.label.unique()

In [ ]:

x_train = data_train['text_stem']
y_train_bin = label_binarizer.transform(y_train)

x_test = data_test['text_stem']
y_test_bin = label_binarizer.transform(y_test)


In [ ]:
batch_size = 32
train_input = tf.data.Dataset.from_tensor_slices((x_train, y_train_bin)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_input = tf.data.Dataset.from_tensor_slices((x_test, y_test_bin)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
train_input


In [ ]:
#%tensorboard --logdir logs/fit

#callbacks = [TensorBoard("logs/fit", histogram_freq=1)]
callbacks = []
history = model.fit(train_input, validation_data=test_input, callbacks=callbacks, epochs=5)

In [ ]:
from reporting.training import plot_history
    
plot_history(history)

In [ ]:
y_predict = label_binarizer.inverse_transform(model.predict(x_test[0:100]))


from reporting.evaluation import plot_confusion_matrix
plot_confusion_matrix(y_test[0:100], y_predict)